# California housing dataset

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

#### References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [10]:
housing = fetch_california_housing()
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [45]:
print(housing.data.shape)
print(housing.target.shape)

(20640, 8)
(20640,)


In [17]:
N, d = housing.data.shape
print("Number of Examples: {}".format(N))
print("Number of features: {}".format(d))

Number of Examples: 20640
Number of features: 8


# Using the Normal Equation

In [19]:
# Adding bias weight as 1 for every examples
housing_data_plus_bias = np.c_[np.ones((N, 1)), housing.data] 

print(np.shape(housing_data_plus_bias))
print(np.shape(housing.target))


(20640, 9)
(20640,)


In [21]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

W = tf.matmul(tf.matrix_inverse(tf.matmul(X, X, transpose_a=True)),tf.matmul(X,y,transpose_a = True))

In [22]:
y_predicted = tf.matmul(X, W)
loss = tf.square(y - y_predicted, name='loss')

In [26]:
with tf.Session() as sess:
    W_values, l = sess.run([W, loss])

In [27]:
W_value

array([[ -3.74804688e+01],
       [  4.35684204e-01],
       [  9.33551788e-03],
       [ -1.06575012e-01],
       [  6.43951416e-01],
       [ -4.26173210e-06],
       [ -3.77273560e-03],
       [ -4.26849365e-01],
       [ -4.40734863e-01]], dtype=float32)

In [30]:
total_loss = np.sum(l)
print("MSE: {}".format(total_loss/N))

MSE: 0.5243528854015261


# Using Batch Gradient Descent

In [31]:
reset_graph()

#### First we need to normalize the feature vectors

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((N, 1)), scaled_housing_data] #shape = [exampeles, features] 
scaled_housing_data_plus_bias = np.transpose(scaled_housing_data_plus_bias) #shape = [features, exampeles]

#Now each column represents an example. It is done for our convenience.

print(scaled_housing_data_plus_bias.shape)

(9, 20640)


In [55]:
housing_target_reshaped = np.transpose(housing.target.reshape(-1, 1))
print(housing_target_reshaped.shape)

(1, 20640)


In [60]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_target_reshaped, dtype=tf.float32, name="y")

W = tf.Variable(tf.zeros([d + 1, 1]), name="weights")


In [61]:
y_predicted = tf.matmul(W, X, transpose_a=True, name="predictions")

In [62]:
error = y - y_predicted
mse = tf.reduce_mean(tf.square(error), name="mse")

In [63]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

In [64]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(optimizer)
    
    best_weights = W.eval()

Epoch 0 MSE = 5.61049
Epoch 100 MSE = 0.711395
Epoch 200 MSE = 0.598105
Epoch 300 MSE = 0.577228
Epoch 400 MSE = 0.563226
Epoch 500 MSE = 0.553028
Epoch 600 MSE = 0.545566
Epoch 700 MSE = 0.540093
Epoch 800 MSE = 0.536071
Epoch 900 MSE = 0.533108


In [65]:
print("Weights: ")
print(best_weights)


Weights: 
[[ 2.06855249]
 [ 0.8390094 ]
 [ 0.14736038]
 [-0.23305154]
 [ 0.25675777]
 [ 0.00578232]
 [-0.04194649]
 [-0.6825164 ]
 [-0.65183544]]


# Using Stochastic Gradient Descent

In [72]:
reset_graph()

In [105]:
n_epochs = 10
learning_rate = 0.00001

In [97]:
X = tf.placeholder(tf.float32, shape=(d + 1, 1), name="X")
y = tf.placeholder(tf.float32, name="y")

In [98]:
W = tf.Variable(tf.zeros([d + 1, 1]), name="weights")

In [99]:
y_predicted = tf.matmul(W, X, transpose_a=True, name="predictions")

In [100]:
loss = tf.square(y - y_predicted, name="loss")

In [101]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [102]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        total_loss = 0
        for i in range(N):
            # Session runs train_op and fetch values of loss
            _, l = sess.run([optimizer, loss], feed_dict={X : scaled_housing_data_plus_bias[:,i].reshape(-1,1), y : housing_target_reshaped[:,i]}) 
            total_loss += l
        print('Epoch {0}: {1}'.format(epoch, total_loss/N))
    
    best_weights = W.eval()

Epoch 0: [[ 4.01238585]]
Epoch 1: [[ 2.12409425]]
Epoch 2: [[ 1.29115784]]
Epoch 3: [[ 0.92067605]]
Epoch 4: [[ 0.75361705]]
Epoch 5: [[ 0.67636538]]
Epoch 6: [[ 0.63895583]]
Epoch 7: [[ 0.61938286]]
Epoch 8: [[ 0.60790956]]
Epoch 9: [[ 0.60022199]]


In [103]:
print("Weights: ")
print(best_weights)

Weights: 
[[ 2.02857256]
 [ 0.77524602]
 [ 0.20480497]
 [-0.00392454]
 [ 0.00435625]
 [ 0.01590534]
 [-0.03963242]
 [-0.26966438]
 [-0.24668297]]


# Using Mini Batch Gradient Descent

In [104]:
reset_graph()

In [124]:
n_epochs = 10
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(N / batch_size))

In [125]:
X = tf.placeholder(tf.float32, shape=(d + 1, batch_size), name="X")
y = tf.placeholder(tf.float32, shape=(1, batch_size), name="y")

In [126]:
W = tf.Variable(tf.zeros([d + 1, 1]), name="weights")

In [127]:
y_predicted = tf.matmul(W, X, transpose_a=True, name="predictions")

In [128]:
loss = tf.square(y - y_predicted, name="loss")
mse = tf.reduce_mean(loss, name = "mse")

In [129]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

In [130]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(N, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[:, indices] 
    y_batch = housing_target_reshaped[:, indices] 
    return X_batch, y_batch

In [131]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            _, l = sess.run([optimizer, mse], feed_dict={X: X_batch, y: y_batch})
        print('Epoch {0}: {1}'.format(epoch, l))
        
    best_weights = W.eval()

Epoch 0: 0.4975327253341675
Epoch 1: 0.6461394429206848
Epoch 2: 0.5028530359268188
Epoch 3: 0.6780144572257996
Epoch 4: 0.41529664397239685
Epoch 5: 0.501575231552124
Epoch 6: 0.35384222865104675
Epoch 7: 0.39754387736320496
Epoch 8: 0.6933445930480957
Epoch 9: 0.3760089874267578


In [135]:
print("Weights: ")
print(best_weights)

Weights: 
[[ 2.07018113]
 [ 0.84754699]
 [ 0.12223973]
 [-0.2771506 ]
 [ 0.35288864]
 [ 0.00471049]
 [-0.01249816]
 [-0.85162646]
 [-0.81841028]]
